In [7]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px 
import plotly.graph_objs as go

In [8]:
df = pd.read_excel(r'.\Data\AG_Demo_ihb.xlsx')
df.dropna(inplace=True)

In [10]:

df['call count percentage'] = df.groupby(['Reason', 'state', 'district', 'block_name'])['call count'].transform(lambda x: x / x.sum() * 100)
treemap_chart = px.treemap(df, path=['Reason', 'state', 'district', 'block_name'],
                           values='call count',
                           hover_data=['call count', 'call count percentage'],
                           labels={'call count': 'Call Count'},
                           title='IHB Not interested Reasons',
                           template='plotly')
treemap_chart.update_traces(textinfo='label+text+value+percent entry',marker_line_color='black',
                           marker_line_width=0.5)
treemap_chart.update_layout(
   uniformtext=dict(mode=False, minsize=8,),
)
treemap_chart.write_html('./templates/treemap.html')